In [1]:
import os
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

e:\LangChain\LangChain\End-End GenAI_Ecommerce_Store_App\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [112]:
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
lllm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
response=lllm.invoke("what is ai in 10 words?")
print(response)

content='Artificial intelligence:  computer systems mimicking human intelligence.\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-9b4fbe43-554b-48ec-b4db-6fb4bafca427-0' usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}}


In [ ]:
from langchain.utilities import SQLDatabase
import pymysql

user = "root"
password = " "
host = "localhost"
name = "ecommerce"

# Corrected connection string
database = SQLDatabase.from_uri(f"mysql+pymysql://{user}:{password}@{host}/{name}", sample_rows_in_table_info=3)

# Print table information
print(database.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [114]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(lllm , database , verbose=True)
question1 = db_chain("how many white shirts left for nike?")




> Entering new SQLDatabaseChain chain...
how many white shirts left for nike?
SQLQuery:Question: how many white shirts left for nike?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' LIMIT 1;
SQLResult: [(68,)]
Answer:Question: how many white shirts left for nike?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' LIMIT 1;
> Finished chain.


In [115]:
question2 = db_chain("How many adidas small size  Black t-shirts remaining?")



> Entering new SQLDatabaseChain chain...
How many adidas small size  Black t-shirts remaining?
SQLQuery:Question: How many adidas small size  Black t-shirts remaining?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Adidas' AND `size` = 'S' AND `color` = 'Black' LIMIT 5;
SQLResult: 
Answer:Question: How many adidas small size  Black t-shirts remaining?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Adidas' AND `size` = 'S' AND `color` = 'Black' LIMIT 5;
> Finished chain.


In [116]:
question3 = db_chain("if I sell all nike small size black t-shirts, what would be my profit after selling it? ")



> Entering new SQLDatabaseChain chain...
if I sell all nike small size black t-shirts, what would be my profit after selling it? 
SQLQuery:

Question: if I sell all nike small size black t-shirts, what would be my profit after selling it? 
SQLQuery: SELECT SUM(`price`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'S' AND `color` = 'Black';
SQLResult: [(Decimal('26'),)]
Answer:Question: if I sell all nike small size black t-shirts, what would be my profit after selling it? 
SQLQuery: SELECT SUM(`price`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'S' AND `color` = 'Black' LIMIT 5;
> Finished chain.


In [117]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "91"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]

In [118]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e = embeddings.embed_query("how many black color nike i have?")
e[:4]

[-0.03700222447514534,
 -0.053954266011714935,
 -0.062193650752305984,
 -0.057708047330379486]

# Create VectorDB

In [119]:
vectorize = [" ".join(example.values())for example in few_shots]
vectorize[0]

"How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 91"

In [120]:
from langchain.vectorstores import Chroma
vectorstore=Chroma.from_texts(vectorize,embedding=embeddings, metadatas=few_shots)

In [121]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector=SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=3)
example_selector.select_examples({"Question": "How much is the total price of the inventory for all XL-size t-shirts"})

[{'Answer': '22292',
  'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '22292',
  'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '22292',
  'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query'}]

In [122]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
print(_mysql_prompt, PROMPT_SUFFIX)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [123]:
from langchain.prompts import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["Question","SQLQuery","DQLResult","Answer"],
    template="\Question:{Question}\nSQLQuery:{SQLQuery}\nSQLResult:{SQLResult}\nAnswer:{Answer}"
)

In [124]:
from langchain.prompts import FewShotPromptTemplate
few_shot_prompt=FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input","table_info","top_k"],
)

In [125]:
new_chain=SQLDatabaseChain.from_llm(lllm,database,verbose=True,prompt=few_shot_prompt)

In [126]:
new_chain("how many white color nike shirts I have?")



> Entering new SQLDatabaseChain chain...
how many white color nike shirts I have?
SQLQuery:SQLQuery: SELECT COUNT(*) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White'
SQLResult: [(4,)]
Answer:Answer:4
> Finished chain.


{'query': 'how many white color nike shirts I have?', 'result': 'Answer:4'}

In [127]:
new_chain("what is the total profit i get from my S-nike's shirt inventory?")



> Entering new SQLDatabaseChain chain...
what is the total profit i get from my S-nike's shirt inventory?
SQLQuery:SQLQuery:SELECT sum(`price` * `stock_quantity`) AS `Total Profit` FROM `t_shirts` WHERE `size` = 'S' AND `brand` = 'Nike'
SQLResult: [(Decimal('3758'),)]
Answer:Answer:3758
> Finished chain.


{'query': "what is the total profit i get from my S-nike's shirt inventory?",
 'result': 'Answer:3758'}